Note : このノートブックでは **PyTorch 2.0 Python 3.10 GPU カーネル と g4dn.xlarge インスタンス** を使ってください。

# ユースケース
このラボでは、合成衛星画像を生成します。これらの画像は研究のために使われたり、画像認識モデルを作成するときの入力画像として利用されます。

# Stable Diffusion

## なぜ Stable Diffusion を Fine-tune するのか？

Stable Diffusion は画像生成において優れていますが、特定の分野に特化した画像の質はあまり高くないかもしれません。たとえば、このノートブックでは、衛星画像を生成しようとします。デフォルトで生成される衛生画像は、いくつかの特徴（高速道路など）をよく表していますが、高速道路を含む衛生画像の品質を向上させるために、実際の衛生画像を用いて Stable Diffusion を Fine-tuning します。

## Fine-tuneの方法

Stable Diffusion を Fine-tune するために、[こちら](https://dreambooth.github.io/) で説明のある DtreamBooth という方法を使います。以下は、DreamBooth の論文の簡単な説明です。
> 私たちの方法では、被写体（例えば特定の犬）の数枚の画像（実験において、通常 3 ~ 5 枚の画像で十分です）と、対応するクラス名（例えば"犬"）を入力とし、固有の被写体に関する一意の識別子をエンコードする Fine-tuneされた/パーソナライズされた text-to-imageモデル を得ます。推論では、異なる文章による一意の識別子を異なるコンテキストの被写体の合成に埋め込むことができます。

**さあ、はじめましょう!**
ハードウェアに関するステップを最初に行います。ノートブックの最初に記載している、正しいカーネルとインスタンスのサイズが選択されていることを確認してください。


In [ ]:
!nvidia-smi

次に、このノートブックで必要ないくつかのライブラリをインストールします。

In [ ]:
!pip install transformers accelerate>=0.16.0 ftfy tensorboard Jinja2 huggingface_hub wandb kaggle git+https://github.com/huggingface/diffusers

### データセット
このチュートリアルのために、Sentinal 2 Satellite 画像からなる、土地利用のための分類データセットである EuroSAT データセットを使います。生成する衛星画像のタイプとして、`Highway`クラスを使用します。`Forest` と `Industrial` クラスは、モデルが `Highway` *インスタンス*を分離する*クラス*として機能します。
ノート: このエクササイズでは、EuroSATデータセットの画像サイズに合わせて、全ての画像を 64,64 にリサイズして表示します。

In [ ]:
!mkdir -p EuroSAT/Highway
!unzip -q eurosat-dataset.zip "EuroSAT/Highway/*" -d ""

In [ ]:
!mkdir -p EuroSAT/base/Forest
!unzip -q eurosat-dataset.zip "EuroSAT/Forest/*" -d "base"

In [ ]:
!mkdir -p EuroSAT/base/Industrial
!unzip -q eurosat-dataset.zip "EuroSAT/Industrial/*" -d "base"

## データセットの観察
EuroSAT データセットの `Highway` クラスのデータを見てみましょう。

In [ ]:
from PIL import Image

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

In [ ]:
actual_img = [Image.open("EuroSAT/Highway/Highway_{}.jpg".format(str(i))) for i in range(1,11)]
image_grid([x.resize((64,64)) for x in actual_img], 2,5)

`Forest` と `Industrial` クラスを見てみましょう。

In [ ]:
actual_img = [Image.open("base/EuroSAT/Forest/Forest_{}.jpg".format(str(i))) for i in range(1,11)]
image_grid([x.resize((64,64)) for x in actual_img], 2,5)

In [ ]:
actual_img = [Image.open("base/EuroSAT/Industrial/Industrial_{}.jpg".format(str(i))) for i in range(1,11)]
image_grid([x.resize((64,64)) for x in actual_img], 2,5)

In [ ]:
import shutil, os
forest_files = os.listdir("base/EuroSAT/Forest")
industrial_files = os.listdir("base/EuroSAT/Industrial")

In [ ]:
!mkdir -p "base/class"

準備のため、Fine-tuningに使用できるパスにファイルをコピーします。

In [ ]:
for filename in forest_files:
    shutil.copyfile(
        os.path.join("base/EuroSAT/Forest",filename),
        os.path.join("base/class",filename)
    )
for filename in industrial_files:
    shutil.copyfile(
        os.path.join("base/EuroSAT/Industrial",filename),
        os.path.join("base/class",filename)
    )

## Stable Diffusion を利用した画像の生成
Fine-tuningを始める前に、Stable Diffusion がデフォルトで生成する画像を見ておきます。Stable Diffusion (1.5) を使って、`Highway` クラスの衛星画像を生成します。
Haggingface の [Diffusers](https://huggingface.co/docs/diffusers/index) ライブラリを利用します。

In [ ]:
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
import torch

pipe = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.to("cuda")

In [ ]:
img_list = pipe(["Sentinel 2 satellite image of a highway"]*10, num_inference_steps=25).images

In [ ]:
image_grid([x.resize((64,64)) for x in img_list], 2,5)

In [ ]:
import gc
from numba import cuda
del(pipe)
gc.collect()
torch.cuda.empty_cache()

# device = cuda.get_current_device()
# device.reset()

## EuroSAT の実際の高速道路画像

In [ ]:
actual_img = [Image.open("EuroSAT/Highway/Highway_{}.jpg".format(str(i))) for i in range(1,11)]
image_grid([x.resize((64,64)) for x in actual_img], 2,5)

Stable Diffustion が直接生成した画像と実際の EuroSAT データセットの画像とで、色やスタイルに大きな違いがあることがわかります。

## DreamBooth と LoRA を利用した Stable Diffution の Fine-tune
正しいタイプの衛星画像をどのように生成するのか学ぶため、text-to-image モデルを Fine-tune します。そのために、2 つの最近のイノベーションである、Dreambooth と LoRA を利用します。
DreamBooth はモデルがより大きな `class` と関連する明確なキャラクターをもった `instance` に適合した画像を生成することをモデルが学習するための新しい方法です。
Low rank adapters (LoRA) は学習するパラメータを大きく減少することで、モデル学習を高速にします。
[こちら](https://github.com/huggingface/diffusers/blob/main/examples/dreambooth/README.md) に役に立つスクリプトがあります。

Stable Diffusion が新しい`instance`を学べるように、一意で（そして短い）トークン/単語を、新しい`instance`を表現するために使います。ここでは、文字順序として、他の意味のある単語と似ておらず、トークン/単語によく使われる `sks` を使います。`sks` は Stable Diffusion の Fine-tuningのチュートリアルでもよく使われます。

最初に、diffusers ライブラリをインストールします。

In [ ]:
!wget https://raw.githubusercontent.com/huggingface/diffusers/main/examples/dreambooth/train_dreambooth_lora.py

次に、Fine-tuningのコードを事項します。Fine-tuningがこのノートブックのローカルで実行されます。
[accelerate](https://github.com/huggingface/accelerate) ライブラリを使うと、PyTorchのコードを複数のGPUで簡単に実行できます。

In [ ]:
!accelerate launch train_dreambooth_lora.py \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5"  \
  --instance_data_dir="EuroSAT/Highway" \
  --output_dir=trained_model \
  --instance_prompt="Sentinel 2 satellite image of sks" \
  --resolution=256 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --checkpointing_steps=100 \
  --learning_rate=1e-4 \
  --report_to="tensorboard" \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --with_prior_preservation \
  --class_data_dir="base/class" \
  --class_prompt="Sentinel 2 satellite image" \
  --max_train_steps=800 \
  --seed="0" 

## 結果の可視化
モデルが学習できたので、次の比較を行います。
1. Fine-tuningせずに、Stable Diffusion で生成した画像
2. LoRA と Dream Booth によって Fine-tuning された Stable Diffusion で生成した画像
3. EuroSATのオリジナル画像

In [ ]:
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
import torch

Fine-tuningなしで生成した画像を見てみましょう。

In [ ]:
pipe = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.to("cuda")

In [ ]:
img_list = pipe(["Sentinel 2 satellite image of a highway"]*3, num_inference_steps=25).images
image_grid([x.resize((128,128)) for x in img_list], 1,3)

次に、Fine-tuningした後の生成画像を見てみます。

In [ ]:
pipe.unet.load_attn_procs("./trained_model/checkpoint-800")

In [ ]:
img_list = pipe(["Sentinel 2 satellite image of sks"]*3, num_inference_steps=25).images

In [ ]:
image_grid([x.resize((128,128)) for x in img_list], 1,3)

最後に、オリジナルの画像を見てみます。

In [ ]:
from PIL.ImageOps import exif_transpose
actual_img = [exif_transpose(Image.open("EuroSAT/Highway/Highway_{}.jpg".format(str(i)))) for i in range(1,4)]
image_grid([x.resize((128,128)) for x in actual_img], 1,3)

これで、このノートブックは終了です。このノートブックでは、画像を利用して Stable Diffusion を Fine-tunning することで、どの程度生成画像の品質が向上するのか見てきました。

## クリーンアップ
このノートブックを閉じた後、左にある白い円の中に黒の四角があるアイコンを使って、インスタンスを停止してください。